In [17]:
!pip install nltk

    100% |████████████████████████████████| 1.4MB 1.3MB/s eta 0:00:01
  Running setup.py bdist_wheel for nltk ... done
  Stored in directory: /home/nora/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [15]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import json

In [18]:
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize

# 1. Load Data

In [19]:
def load_jsonline(filename, limit):
    data = []
    with open(filename) as f:
        counter = 0
        for line in f:
            counter += 1
            py_obj = json.loads(line)
            data.append(py_obj)
            if counter > limit:
                break
    return data

In [4]:
nlp = pipeline("sentiment-analysis")

In [10]:
print(nlp("She is a doctor"))
print(nlp("He is a doctor"))

[{'label': 'POSITIVE', 'score': 0.994561}]
[{'label': 'POSITIVE', 'score': 0.9951726}]


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

In [13]:
classes = ["paraphrase", "no paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")

Should be paraphrase
paraphrase: 10%
no paraphrase: 90%

Should not be paraphrase
paraphrase: 94%
no paraphrase: 6%
